# Cleaning data
**Handling missing values, outliers, duplicates, data integrity and standardizing data**

**Import** data from "data/raw.csv"

**Export** to "data/cleaned.csv"

In [2]:
# Setup

import numpy as np
import pandas as pd

from pkg import utils
from pkg import clean

In [3]:
run_save = True
imputing = True
encoding = True
standarding = False
one_hot_encoding = True

In [4]:
# Importing data

parent = 'data'
filename = 'raw.csv'

path = utils.get_path(parent,filename)

df = pd.read_csv(path, sep=";", index_col=0)

In [5]:
raw_df = df

raw_df.head()

,TIPO_RELACIONAMENTO,ESTADO_LOJA,CATEGORIA_PRODUTO,RENDA_ESPERADA,ESTRATEGIA_VENDAS,AVAL_SATISFACAO,VL_LIMITE_CREDITO_INICIAL_BANCO,REGIAO_LOJA,VL_FINANCEIRO_BCB,VL_OPERACAO_CARTAO_CREDITO,VL_LIMITE_CREDITO_BCB,PONTUACAO_DESEMPENHO,TEMPO_EXISTENCIA_ONLINE,AVALIACAO_RISCO,SCORE_INTERNO,RENDA
ID_LOJA,,,,,,,,,,,,,,,,
922686,Tipo 3,RN,Alimentos e Bebidas,2199000,Nao informado,7.31,60000,NE,4265800,3546400,719400,792.0,37,300.0,528.0,242400
938287,Tipo 2,BA,Saude e Beleza,300000,Personalizada,3.57,50000,NE,168700,49500,500,392.0,22,300.0,NaN,235100
939120,Tipo 3,CE,Alimentos e Bebidas,650000,Padrao,5.08,40000,NE,125900,90900,35000,653.0,27,300.0,NaN,420600
933554,Tipo 2,SP,Saude e Beleza,300000,Nao informado,6.29,160000,SE,1417900,379200,18900,579.0,33,150.0,NaN,340600
948294,Tipo 3,SP,Alimentos e Bebidas,1200000,Personalizada,9.23,60000,SE,402000,206800,195100,792.0,30,300.0,NaN,456900


In [8]:
numerical_variables = df.select_dtypes(include=['number'])
numerical_variables = numerical_variables.columns

categorical_variables = df.select_dtypes(exclude=['number'])
categorical_variables = categorical_variables.columns

print(f'Numerical: \n{numerical_variables}\n')
print(f'Categorical: \n{categorical_variables}')

Numerical: 
Index(['RENDA_ESPERADA', 'AVAL_SATISFACAO', 'VL_LIMITE_CREDITO_INICIAL_BANCO',
       'VL_FINANCEIRO_BCB', 'VL_OPERACAO_CARTAO_CREDITO',
       'VL_LIMITE_CREDITO_BCB', 'PONTUACAO_DESEMPENHO',
       'TEMPO_EXISTENCIA_ONLINE', 'AVALIACAO_RISCO', 'SCORE_INTERNO', 'RENDA'],
      dtype='object')

Categorical: 
Index(['TIPO_RELACIONAMENTO', 'ESTADO_LOJA', 'CATEGORIA_PRODUTO',
       'ESTRATEGIA_VENDAS', 'REGIAO_LOJA'],
      dtype='object')


In [6]:
# df['ID_LOJA'] = df['ID_LOJA'].astype(str)

# Missing values

In [7]:
missing_values = df.isnull().sum()

print("Columns with missing values:")
for column_name, num_missing in missing_values.items():
    if num_missing > 0:
        print(f"{column_name}\n\tNumber of missing values: {num_missing}")

Columns with missing values:
AVAL_SATISFACAO
	Number of missing values: 10
PONTUACAO_DESEMPENHO
	Number of missing values: 29
AVALIACAO_RISCO
	Number of missing values: 314
SCORE_INTERNO
	Number of missing values: 23613


In [8]:
# Impute missing values

# Imputing under not MNAR assumption (Missing not at random)

if imputing:
    from sklearn.impute import SimpleImputer

    df = clean.random_sample_imputation(df)

# Categorical

In [9]:
df.head()

,TIPO_RELACIONAMENTO,ESTADO_LOJA,CATEGORIA_PRODUTO,RENDA_ESPERADA,ESTRATEGIA_VENDAS,AVAL_SATISFACAO,VL_LIMITE_CREDITO_INICIAL_BANCO,REGIAO_LOJA,VL_FINANCEIRO_BCB,VL_OPERACAO_CARTAO_CREDITO,VL_LIMITE_CREDITO_BCB,PONTUACAO_DESEMPENHO,TEMPO_EXISTENCIA_ONLINE,AVALIACAO_RISCO,SCORE_INTERNO,RENDA
ID_LOJA,,,,,,,,,,,,,,,,
922686,Tipo 3,RN,Alimentos e Bebidas,2199000,Nao informado,7.31,60000,NE,4265800,3546400,719400,792.0,37,300.0,528.0,242400
938287,Tipo 2,BA,Saude e Beleza,300000,Personalizada,3.57,50000,NE,168700,49500,500,392.0,22,300.0,707.0,235100
939120,Tipo 3,CE,Alimentos e Bebidas,650000,Padrao,5.08,40000,NE,125900,90900,35000,653.0,27,300.0,462.0,420600
933554,Tipo 2,SP,Saude e Beleza,300000,Nao informado,6.29,160000,SE,1417900,379200,18900,579.0,33,150.0,600.0,340600
948294,Tipo 3,SP,Alimentos e Bebidas,1200000,Personalizada,9.23,60000,SE,402000,206800,195100,792.0,30,300.0,505.0,456900


In [10]:
# Categorical to numerical

if encoding:
    from sklearn.preprocessing import LabelEncoder
    
    label_mapping = {}
    
    for col in categorical_variables:
        label_encoder = LabelEncoder()
        df[col] = label_encoder.fit_transform(df[col])
        label_mapping[col] = dict(zip(label_encoder.transform(label_encoder.classes_), label_encoder.classes_))

    print(label_mapping)

    #df[categorical_variables] = df[categorical_variables].apply(label_encoder.fit_transform)
    #df[categorical_variables]

    
    

{'TIPO_RELACIONAMENTO': {0: 'Tipo 1', 1: 'Tipo 2', 2: 'Tipo 3', 3: 'Tipo 4', 4: 'Tipo 5'}, 'ESTADO_LOJA': {0: 'AC', 1: 'AL', 2: 'AM', 3: 'AP', 4: 'BA', 5: 'CE', 6: 'DF', 7: 'ES', 8: 'GO', 9: 'MA', 10: 'MG', 11: 'MS', 12: 'MT', 13: 'PA', 14: 'PB', 15: 'PE', 16: 'PI', 17: 'PR', 18: 'RJ', 19: 'RN', 20: 'RO', 21: 'RR', 22: 'RS', 23: 'SC', 24: 'SE', 25: 'SP', 26: 'TO'}, 'CATEGORIA_PRODUTO': {0: 'Alimentos e Bebidas', 1: 'Eletronicos', 2: 'Moda e Vestuário', 3: 'Outros', 4: 'Produtos para Casa', 5: 'Saude e Beleza'}, 'ESTRATEGIA_VENDAS': {0: 'Nao informado', 1: 'Padrao', 2: 'Personalizada'}, 'REGIAO_LOJA': {0: 'CO', 1: 'N', 2: 'NE', 3: 'S', 4: 'SE'}}


In [11]:
df.head()

,TIPO_RELACIONAMENTO,ESTADO_LOJA,CATEGORIA_PRODUTO,RENDA_ESPERADA,ESTRATEGIA_VENDAS,AVAL_SATISFACAO,VL_LIMITE_CREDITO_INICIAL_BANCO,REGIAO_LOJA,VL_FINANCEIRO_BCB,VL_OPERACAO_CARTAO_CREDITO,VL_LIMITE_CREDITO_BCB,PONTUACAO_DESEMPENHO,TEMPO_EXISTENCIA_ONLINE,AVALIACAO_RISCO,SCORE_INTERNO,RENDA
ID_LOJA,,,,,,,,,,,,,,,,
922686,2,19,0,2199000,0,7.31,60000,2,4265800,3546400,719400,792.0,37,300.0,528.0,242400
938287,1,4,5,300000,2,3.57,50000,2,168700,49500,500,392.0,22,300.0,707.0,235100
939120,2,5,0,650000,1,5.08,40000,2,125900,90900,35000,653.0,27,300.0,462.0,420600
933554,1,25,5,300000,0,6.29,160000,4,1417900,379200,18900,579.0,33,150.0,600.0,340600
948294,2,25,0,1200000,2,9.23,60000,4,402000,206800,195100,792.0,30,300.0,505.0,456900


In [ ]:
if one_hot_encoding:
    df = pd.get_dummies(df, columns=categorical_variables, drop_first=True).astype('int')

    y = df.pop("RENDA")
    df.insert(len(df.columns), "RENDA", y)

In [ ]:
# Standardizing numerical values

if standarding:

    from sklearn.preprocessing import StandardScaler

    scaler = StandardScaler()
    df[numerical_variables] = scaler.fit_transform(df[numerical_variables])

df[numerical_variables]

In [ ]:
# Exporting data

cleaned = df # Add final df

if run_save:
    utils.save_df(cleaned, "cleaned.csv")